In [1]:
import pandas as pd
import json
from IPython.display import display, HTML

pd.set_option('display.max_colwidth', 500)

CSV_PATH = '../tpch_with_cost_requests'
BENCHMARK = 'tpch'

In [2]:
class AlgorithmStep():
    def __init__(self, indexes, number_of_indexes, step, memory_consumption, workload_processing_cost):
        self.indexes = indexes
        self.number_of_indexes = number_of_indexes
        self.step = step
        self.memory_consumption = memory_consumption
        self.workload_processing_cost = workload_processing_cost
        
    def memory_consumption_gb(self):
        return f"{self.memory_consumption / 1E9:.2f}"
    
    def __repr__(self):
        return f"At step: {self.step} with {self.number_of_indexes} indexes ({self.memory_consumption_gb()} GB):\nCost: {self.workload_processing_cost}\nIndexes: {self.indexes}"
        
def calculate_cost(columns, row):
    cost = 0
    for column in columns:
        if column[0] == 'q':
            cost += float(json.loads(row[column])['Cost'])
    
    return cost

def shorten_tbl_name(index_name):
    # CAREFUL THIS COMPLETELY REMOVES nation
    if 'nation.' in index_name:
        return None
    return index_name.replace('nation.', 'n.').replace('customer.', 'c.').replace('lineitem.', 'l.').replace('partsupp.', 'ps.').replace('orders.', 'o.').replace('supplier.', 's.')

def cut_index_string(indexes):
    result = []
    removed_brackets = indexes[1:-1]
    for split_1 in removed_brackets.split('I('):
        without_brace = split_1.split(')')[0]
        if without_brace == '':
            continue
        index_name = f"[{without_brace.replace('C ', '')}]"
        index_name = shorten_tbl_name(index_name)
        if index_name is None:
            continue
        result.append(index_name)
        
    return result

def get_new_and_removed_indexes_str(current_indexes, previous_indexes):
    removed_indexes = previous_indexes - current_indexes
    new_indexes = current_indexes - previous_indexes
    
    new_index_str = '\n'.join(new_indexes) if len(new_indexes) else '-'
    removed_index_str = '\n'.join(removed_indexes) if len(removed_indexes) else '-'
    
    return new_index_str, removed_index_str

def get_cost(columns, row, previous_cost, initial_cost):
    cost = calculate_cost(columns, row)
    cost_relative_to_previous = cost / previous_cost * 100
    cost_relative_to_initial = cost / initial_cost * 100
    cost_string = f'{cost_relative_to_previous:.2f}% / {cost_relative_to_initial:.2f}%'
    
    return cost_string, cost

In [3]:
df = pd.read_csv(f'{CSV_PATH}/results_no_index_{BENCHMARK}_22_queries.csv', sep=';')
df = df.drop(['q20'],axis=1)
initial_cost = calculate_cost(df.columns, df.iloc[0])
print(f'Initial cost: {initial_cost}')

Initial cost: 94942316.34


# Budget Table

In [6]:
algorithm_names = sorted(['epic', 'ibm', 'drop', 'microsoft'])
df_table = pd.DataFrame(columns=['Max Budget (GB)', 'Metric'] + algorithm_names)
metrics = ['New Indexes', 'Removed Indexes', '% of previous / initial cost']
last_steps_indexes_per_algo = {}
previous_cost = {}
for algorithm_name in algorithm_names:
    last_steps_indexes_per_algo[algorithm_name] = set()
    previous_cost[algorithm_name] = initial_cost
    
df_data_ibm = pd.read_csv(f'{CSV_PATH}/results_ibm-max2_{BENCHMARK}_22_queries.csv',sep=';')
df_data_ibm = df_data_ibm.drop(['q20'],axis=1)
df_data_epic = pd.read_csv(f'{CSV_PATH}/results_epic_max_indexes_{BENCHMARK}_22_queries.csv',sep=';')
df_data_epic = df_data_epic.drop(['q20'],axis=1)
df_data_drop = pd.read_csv(f'{CSV_PATH}/results_drop_heuristic_{BENCHMARK}_22_queries.csv',sep=';')
df_data_drop = df_data_drop.drop(['q20'],axis=1)
df_data_ms = pd.read_csv(f'{CSV_PATH}/results_microsoft-max2_{BENCHMARK}_22_queries.csv',sep=';')
df_data_ms = df_data_ms.drop(['q20'],axis=1)


for (_, row_epic), (__, row_ibm) in zip(df_data_epic.iterrows(), df_data_ibm.iterrows()):
    budget = json.loads(row_ibm['parameters'])['budget']
    budget_bytes = budget * 1000 * 1000
    for idx, metric in enumerate(metrics):
        df_table = df_table.append(pd.DataFrame([[budget / 1000 if idx == 0 else '', metric] + len(algorithm_names) * ['-']], columns=df_table.columns),ignore_index=True)
    
    # EPIC
    current_indexes = set(cut_index_string(row_epic['indexed columns']))    
    new_index_str, removed_index_str = get_new_and_removed_indexes_str(current_indexes, last_steps_indexes_per_algo['epic'])
    df_table['epic'].values[-3] = new_index_str
    df_table['epic'].values[-2] = removed_index_str
    last_steps_indexes_per_algo['epic'] = current_indexes
    
    cost_string, cost = get_cost(df_data_epic.columns, row_epic, previous_cost['epic'], initial_cost)
    previous_cost['epic'] = cost
    df_table['epic'].values[-1] = cost_string
    
    # IBM
    current_indexes = set(cut_index_string(row_ibm['indexed columns']))    
    new_index_str, removed_index_str = get_new_and_removed_indexes_str(current_indexes, last_steps_indexes_per_algo['ibm'])
    df_table['ibm'].values[-3] = new_index_str
    df_table['ibm'].values[-2] = removed_index_str
    last_steps_indexes_per_algo['ibm'] = current_indexes
    
    cost_string, cost = get_cost(df_data_ibm.columns, row_ibm, previous_cost['ibm'], initial_cost)
    previous_cost['ibm'] = cost
    df_table['ibm'].values[-1] = cost_string
    
    #DROP
    relevant = df_data_drop[(df_data_drop['memory consumption'] < budget_bytes)].tail(1)
    if relevant.shape[0] > 0:
        row_drop = relevant.iloc[0]
        current_indexes = set(cut_index_string(row_drop['indexed columns']))
        new_index_str, removed_index_str = get_new_and_removed_indexes_str(current_indexes, last_steps_indexes_per_algo['drop'])
        df_table['drop'].values[-3] = new_index_str
        df_table['drop'].values[-2] = removed_index_str
        last_steps_indexes_per_algo['drop'] = current_indexes
    
        cost_string, cost = get_cost(df_data_ibm.columns, row_ibm, previous_cost['drop'], initial_cost)
        previous_cost['drop'] = cost
        df_table['drop'].values[-1] = cost_string
        
    #MS
    relevant = df_data_ms[(df_data_ms['memory consumption'] < budget_bytes)].tail(1)
    if relevant.shape[0] > 0:
        row_ms = relevant.iloc[0]
        current_indexes = set(cut_index_string(row_ms['indexed columns']))
        new_index_str, removed_index_str = get_new_and_removed_indexes_str(current_indexes, last_steps_indexes_per_algo['microsoft'])
        df_table['microsoft'].values[-3] = new_index_str
        df_table['microsoft'].values[-2] = removed_index_str
        last_steps_indexes_per_algo['microsoft'] = current_indexes
    
        cost_string, cost = get_cost(df_data_ibm.columns, row_ibm, previous_cost['microsoft'], initial_cost)
        previous_cost['microsoft'] = cost
        df_table['microsoft'].values[-1] = cost_string
    
    
        
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
new_df = pretty_print(df_table)
new_df

,Max Budget (GB),Metric,drop,epic,ibm,microsoft
0,0.2,New Indexes,-,[ps.ps_suppkey],"[part.p_partkey,part.p_name][s.s_name][s.s_suppkey,s.s_nationkey][s.s_suppkey,s.s_name][s.s_nationkey,s.s_suppkey]",-
1,,Removed Indexes,-,-,-,-
2,,% of previous / initial cost,-,98.78% / 98.78%,98.91% / 98.91%,-
3,0.4,New Indexes,-,[l.l_partkey],[ps.ps_suppkey],-
4,,Removed Indexes,-,-,-,-
5,,% of previous / initial cost,-,35.70% / 35.26%,99.21% / 98.13%,-
6,0.8,New Indexes,-,"[l.l_partkey,l.l_suppkey]","[c.c_custkey][part.p_size][part.p_partkey,part.p_container][c.c_mktsegment,c.c_custkey][part.p_container,part.p_partkey]",-
7,,Removed Indexes,-,[l.l_partkey],-,-
8,,% of previous / initial cost,-,98.04% / 34.57%,99.82% / 97.95%,-
9,1.2,New Indexes,-,[l.l_orderkey],"[part.p_type,part.p_partkey][ps.ps_partkey][c.c_nationkey,c.c_custkey]",-


# Step Table

In [ ]:
algorithm_names = sorted(['epic_max_indexes', 'microsoft-max2', 'ibm_max_indexes', 'drop_heuristic'])
# algorithm_names = sorted(['epic_max_indexes', 'ibm_max_indexes', 'drop_heuristic'])

algorithm_steps = {}

for algorithm_name in algorithm_names:
    path = f'{CSV_PATH}/results_{algorithm_name}_{BENCHMARK}_22_queries.csv'
    df = pd.read_csv(path,sep=';')
    df = df.drop(['q20'],axis=1)
    algorithm_steps[algorithm_name] = []
    for step, row in df.iterrows():
        cost = calculate_cost(df.columns, row)
        ast = AlgorithmStep(row['indexed columns'], row['#indexes'], step, row['memory consumption'], cost)
        algorithm_steps[algorithm_name].append(ast)

#     print(algorithm_name)
#     for step in algorithm_steps[algorithm_name]:
#         print(step)
#         print(f'Cost relative to no indexes: {step.workload_processing_cost / initial_cost * 100}%')
#         print()
#     print()
#     print()
#     print()

In [122]:
df = pd.DataFrame(columns=['Step #', 'Metric'] + algorithm_names)
metrics = ['New Indexes', 'Removed Indexes', '% of previous / initial cost', 'Total Size (GB)']
most_steps = max(list(map(lambda x: len(algorithm_steps[x]), algorithm_steps)))
last_steps_indexes_per_algo = {}

previous_cost = {}
for algorithm_name in algorithm_names:
    previous_cost[algorithm_name] = initial_cost

for step in range(0, most_steps):
    for idx, metric in enumerate(metrics):
        df = df.append(pd.DataFrame([[step + 1 if idx == 0 else '', metric] + len(algorithm_names) * ['-']], columns=df.columns),ignore_index=True)
    for algorithm_name in algorithm_names:
        if algorithm_name not in last_steps_indexes_per_algo:
            last_steps_indexes_per_algo[algorithm_name] = set()
        if step < len(algorithm_steps[algorithm_name]):
            algorithm_step = algorithm_steps[algorithm_name][step]
            df[algorithm_name].values[-1] = algorithm_step.memory_consumption_gb()
            cost_relative_to_previous = algorithm_step.workload_processing_cost / previous_cost[algorithm_name] * 100
            previous_cost[algorithm_name] = algorithm_step.workload_processing_cost
            cost_relative_to_initial = algorithm_step.workload_processing_cost / initial_cost * 100
            cost_string = f'{algorithm_step.workload_processing_cost} ({cost_relative_to_previous:.2f}%)'
            cost_string = f'{cost_relative_to_previous:.2f}% / {cost_relative_to_initial:.2f}%'
            df[algorithm_name].values[-2] = cost_string
            
            indexes = set(cut_index_string(algorithm_step.indexes))
            removed_indexes = last_steps_indexes_per_algo[algorithm_name] - indexes
            new_indexes = indexes - last_steps_indexes_per_algo[algorithm_name]
            last_steps_indexes_per_algo[algorithm_name] = indexes
#             if 'drop' in algorithm_name:
#                 print(indexes)
#                 print(new_index)
#                 print()
#             assert len(new_index) == 1, f'There should be exactly one new index: {step} {algorithm_name}'
#             already_seen_indexes_per_algo[algorithm_name] |= (new_index)
            new_index_str = '\n'.join(new_indexes) if len(new_indexes) else '-'
#             new_index_str = next(iter(new_index))
#             new_index_str = new_index_str.replace(',', '\n')
            df[algorithm_name].values[-4] = new_index_str
            removed_index_str = '\n'.join(removed_indexes) if len(removed_indexes) else '-'
#             removed_index_str = removed_index_str.replace(',', '\n')
            df[algorithm_name].values[-3] = removed_index_str
            
            
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
new_df = pretty_print(df.head(15 * len(metrics)))
new_df

,Step #,Metric,drop_heuristic,epic_max_indexes,ibm_max_indexes,microsoft-max2
0,1,New Indexes,[l.l_partkey],[ps.ps_suppkey],[s.s_name],"[l.l_partkey,l.l_suppkey]"
1,,Removed Indexes,-,-,-,-
2,,% of previous / initial cost,36.43% / 36.43%,98.78% / 98.78%,100.03% / 100.03%,35.57% / 35.57%
3,,Total Size (GB),1.57,0.21,0.01,2.51
4,2,New Indexes,[ps.ps_partkey],[l.l_partkey],"[part.p_partkey,part.p_name]","[ps.ps_suppkey,ps.ps_partkey]"
5,,Removed Indexes,-,-,-,-
6,,% of previous / initial cost,92.91% / 33.85%,35.70% / 35.26%,98.96% / 98.99%,94.29% / 33.54%
7,,Total Size (GB),1.78,1.78,0.16,2.84
8,3,New Indexes,[l.l_suppkey],"[l.l_partkey,l.l_suppkey]","[ps.ps_suppkey,ps.ps_availqty,ps.ps_partkey]","[l.l_suppkey,l.l_orderkey]"
9,,Removed Indexes,-,[l.l_partkey],-,-


In [ ]:
import html2markdown
html2markdown.convert(df.to_html(index=False).replace('\n', ''))

In [47]:
print(df_table.to_latex(index=False,escape=True).replace('\\textbackslash n', ' '))

\begin{tabular}{llllll}
\toprule
Max Budget (GB) &                        Metric &                                                                                                                                                 drop &                                                     epic &                                                                                                                                                                                                           ibm &                                                                                                                                                                                                                microsoft \\
\midrule
            0.2 &                   New Indexes &                                                                                                                                                    - &                                                        - &           